In [46]:
import pandas as pd
import random
import string

In [47]:
def generate_alphabet_tuples():
    return [(chr(i), []) for i in range(ord('a'), ord('z')+1)]


In [48]:
def generate_character_mapping():
    alphabet_tuples = generate_alphabet_tuples()
    for i in range(8):
        ascii_low = string.ascii_lowercase
        for t in alphabet_tuples:
            c =  random.choice(ascii_low)
            t[1].append(c)
            ascii_low =ascii_low.replace(c,'')
    
    
    return alphabet_tuples

In [49]:
alphabet_tuples = generate_character_mapping()

In [50]:
alphabet_tuples

[('a', ['k', 'x', 'p', 'p', 'q', 'q', 'm', 'z']),
 ('b', ['d', 'q', 'z', 'h', 'n', 'm', 'c', 'f']),
 ('c', ['u', 'y', 'c', 'e', 'c', 'z', 'j', 'o']),
 ('d', ['r', 'u', 'h', 'b', 'z', 'c', 'u', 'j']),
 ('e', ['f', 's', 's', 's', 'y', 'b', 'e', 'u']),
 ('f', ['h', 'g', 'g', 'n', 'd', 'a', 'q', 'v']),
 ('g', ['j', 'o', 'm', 'w', 'j', 'e', 'l', 'c']),
 ('h', ['q', 'b', 'j', 'd', 'r', 'g', 'w', 'n']),
 ('i', ['t', 'i', 'f', 'x', 'm', 't', 't', 'l']),
 ('j', ['m', 'c', 'e', 'k', 'a', 'o', 'i', 'g']),
 ('k', ['l', 'v', 'u', 'r', 'l', 's', 'd', 'h']),
 ('l', ['a', 'z', 'v', 'f', 'b', 'x', 'r', 'k']),
 ('m', ['g', 'j', 'l', 'v', 't', 'n', 'v', 'i']),
 ('n', ['s', 'l', 'a', 'g', 'g', 'y', 'z', 'm']),
 ('o', ['o', 'n', 't', 'a', 'h', 'l', 'k', 'y']),
 ('p', ['n', 'a', 'b', 't', 'i', 'u', 'n', 'a']),
 ('q', ['i', 'e', 'd', 'y', 'v', 'i', 'p', 'b']),
 ('r', ['e', 'p', 'i', 'z', 'o', 'h', 'g', 's']),
 ('s', ['w', 'h', 'o', 'i', 'f', 'd', 'h', 'p']),
 ('t', ['x', 'm', 'y', 'c', 'u', 'k', 's', 't']),


## change to dataframe 

In [51]:
def generate_alphabet_df():
    # dataframe with indexes from 'a' to 'z' and columns 'value_1' to 'value_8'
    alphabet_df = pd.DataFrame(index=[chr(i) for i in range(ord('a'), ord('d')+1)], 
                               columns=['column_'+str(i) for i in range(1,3)])
    alphabet_df.index.name = "carrier_letter"
    
    
    for each_column in alphabet_df.columns:
        ascii_low = string.ascii_lowercase
        for each_index in alphabet_df.index:
            c = random.choice(ascii_low)
            alphabet_df.loc[each_index, each_column] = c
            ascii_low = ascii_low.replace(c,'')
            
    alphabet_df.reset_index(level=0, inplace=True)
            
    return alphabet_df

In [52]:
alphabet_df = generate_alphabet_df()


In [53]:
print(alphabet_df)

  carrier_letter column_1 column_2
0              a        k        p
1              b        d        b
2              c        a        h
3              d        z        y


In [54]:
def shift_single_column(df, column_name, shift_value =1):
    column_to_shift = df[column_name]
    
    lost_values = column_to_shift[-shift_value:]
    
    shifted_column = df[column_name].shift(shift_value)
   
    df[column_name] = pd.concat([lost_values, shifted_column[shift_value:]]).reset_index(drop=True)

    return df


In [55]:
alphabet_df_shifted = shift_single_column(alphabet_df, 'column_1')

In [56]:
alphabet_df_shifted

,carrier_letter,column_1,column_2
0,a,z,p
1,b,k,b
2,c,d,h
3,d,a,y


## shift columns to achieve no repetition in rows

In [57]:
data = {
    'carrier_letter': ['a', 'b', 'c', 'd'],
    'column_1': ['i', 'z', 'l', 'v'],
    'column_2': ['i', 'l', 'w', 'r'],
    'column_3': ['g', 'h', 'o', 'y'],
    'column_4': ['g', 'i', 'k', 'o'],
    'column_5': ['g', 'b', 'c', 'd'],
}

df = pd.DataFrame(data)

In [58]:
df

,carrier_letter,column_1,column_2,column_3,column_4,column_5
0,a,i,i,g,g,g
1,b,z,l,h,i,b
2,c,l,w,o,k,c
3,d,v,r,y,o,d


In [59]:
df.columns[1:]
# get index of the column by name
column_index = df.columns.get_loc('column_1')
column_index

1

In [60]:
# one random column
column_name = random.choice(df.columns[1:])
column_name

'column_4'

In [61]:
def move_columns(df,column, column_to_change):
    """Function shifts the values of a column in one column, so there will be no same characters in the same row.
       Function should return df with columns with no same characters in the same row.
    """
    iter = 0
    was_shifted = False
    
    while any(df[column] == df[column_to_change]):
        print(f"Column {column} has same values as {column_to_change}")
        df = shift_single_column(df, column_to_change)
        was_shifted = True
        iter += 1
        if iter > 100:
            print("Too many iterations, breaking out of the loop.")
            break
    
    return df, was_shifted

In [62]:
def fix_repetition_for_one_column(df,  column_to_change, iter):
    was_shifted = False
    for column in df.columns[1:iter]:
        print(f"Checking column {column} for repetition with {column_to_change}")
        if column == column_to_change: #necessary to skip the column that is being changed
            continue
        df , was_shifted = move_columns(df,column, column_to_change)

    return df, was_shifted

In [63]:
def fix_repetition(df):
    """Function iterates over all columns comparing them and shifting the values; after shift it begins comparing columns from the start"""
    iter = -1*len(df.columns[2:])
    
    was_shifted = True
    # skip carrier_letter and first column and fix repetition in the rest of the columns
    for column_to_change in df.columns[2:]:
        # print(column_to_change)
        while was_shifted:
            was_shifted = False
            df, was_shifted =fix_repetition_for_one_column(df,  column_to_change, iter)
            print(column_to_change)
            
        iter += 1
        was_shifted = True

    return df

In [64]:
print(df)

  carrier_letter column_1 column_2 column_3 column_4 column_5
0              a        i        i        g        g        g
1              b        z        l        h        i        b
2              c        l        w        o        k        c
3              d        v        r        y        o        d


In [65]:
df_fixed = fix_repetition(df)


Checking column column_1 for repetition with column_2
Column column_1 has same values as column_2
Column column_1 has same values as column_2
column_2
Checking column column_1 for repetition with column_2
column_2
Checking column column_1 for repetition with column_3
Checking column column_2 for repetition with column_3
column_3
Checking column column_1 for repetition with column_4
Checking column column_2 for repetition with column_4
Checking column column_3 for repetition with column_4
Column column_3 has same values as column_4
column_4
Checking column column_1 for repetition with column_4
Checking column column_2 for repetition with column_4
Column column_2 has same values as column_4
Checking column column_3 for repetition with column_4
column_4
Checking column column_1 for repetition with column_5
Checking column column_2 for repetition with column_5
Checking column column_3 for repetition with column_5
Column column_3 has same values as column_5
Checking column column_4 for repe

In [66]:
print(df_fixed)

  carrier_letter column_1 column_2 column_3 column_4 column_5
0              a        i        w        g        k        d
1              b        z        r        h        o        g
2              c        l        i        o        g        b
3              d        v        l        y        i        c


In [67]:

num_columns = len(df.columns)
print(num_columns)

6
